In [1]:
import malaya_speech

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[2025-05-18 15:14:27,232] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


In [2]:
# from huggingface_hub import snapshot_download
# snapshot_download(
#     repo_id="mesolitica/Malaysian-Emilia", repo_type = 'dataset',
#     allow_patterns=['sg-podcast_processed-*.zip', 'malaysian-podcast_processed-*.zip'],
#     local_dir = './'
# )

In [3]:
# !wget https://gist.githubusercontent.com/huseinzol05/2e26de4f3b29d99e993b349864ab6c10/raw/9b2251f3ff958770215d70c8d82d311f82791b78/unzip.py
# !python3 unzip.py

In [4]:
from glob import glob
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import json
import os
import re
import itertools
import random
from tqdm import tqdm
from multiprocess import Pool
from collections import defaultdict
import numpy as np

def generate_trigrams(text):
    words = text.split()
    return list(zip(words, words[1:], words[2:]))

def skip_trigrams(text):
    trigrams = generate_trigrams(text)
    count = defaultdict(int)
    total = 0
    for t in trigrams:
        count[''.join(t)] += 1
        total += 1
    if len(count.keys()) < 3:
        return True
    for k, v in count.items():
        if (v / total) > 0.2:
            return True
    return False

def chunks(l, n):
    for i in range(0, len(l), n):
        yield (l[i: i + n], i // n)

def multiprocessing(strings, function, cores=6, returned=True):
    df_split = chunks(strings, len(strings) // cores)
    pool = Pool(cores)
    pooled = pool.map(function, df_split)
    pool.close()
    pool.join()

    if returned:
        return list(itertools.chain(*pooled))

In [5]:
files = glob('*/**/*.json', recursive = True)
len(files)

17556

In [6]:
import malaya_speech

model = malaya_speech.speaker_vector.nemo('huseinzol05/nemo-titanet_large')
model = model.eval()

In [7]:
_ = model.cuda()

In [10]:
rejected = [
    'terima kasih kerana menonton',
    'terima kasih',
    'thank you for watching',
]

def new_path(f):
    return f.replace('_processed/', '_processed_trim_moshi/').replace('.mp3', '.moshi')

def loop(files):
    files, _ = files
    data = []
    for file in tqdm(files):
        folder = os.path.split(file)[0]
        folder_folder = os.path.split(folder)[1]

        try:
            with open(file) as fopen:
                d = json.load(fopen)
        except:
            continue
            
        speakers = defaultdict(dict)
        
        audio_files = []
        index = 0
        for no, obj in enumerate(d):
            text = obj["text"].strip()
            
            rt_ = re.sub('[^a-z ]+', '', text.lower()).strip()
            if any([s == rt_ for s in rejected]):
                continue
                
            split = text.split()
            ones = [w for w in split if len(w) <= 1]
            if (len(ones) / len(split)) >= 0.5:
                continue
                
            if any([(len(set(w)) / len(w)) < 0.3 for w in split]):
                continue
            
            try:
                dense = CountVectorizer(ngram_range = (3,3)).fit_transform([text]).todense()
                repeat = (dense > 3).sum() >= 1
                if repeat:
                    continue
            except:
                continue
            
            audio_path = os.path.join(folder, f'{folder_folder}_{no}.mp3')
            
            if not os.path.exists(audio_path):
                continue
            
            speakers[obj['speaker']][index] = {
                'audio': audio_path,
                'transcription': text,
            }
            audio_files.append(audio_path)
            index += 1
        
        ys = [malaya_speech.load(f)[0] for f in audio_files]
        if not len(ys):
            continue
            
        vectors = []
        for i in range(0, len(ys), 4):
            vectors_ = model(ys[i: i + 4])
            vectors.append(vectors_)
            
        cosine = cosine_similarity(np.concatenate(vectors))
        
        for speaker in speakers.keys():
            data_ = []
            for row in speakers[speaker]:
                for row_ in speakers[speaker]:
                    if row == row_:
                        continue
                    
                    if cosine[row, row_] < 0.8:
                        continue

                    data_.append({
                        'reference_audio': speakers[speaker][row]['audio'],
                        'reference_text': speakers[speaker][row]['transcription'],
                        'target_audio': speakers[speaker][row_]['audio'],
                        'target_text': speakers[speaker][row_]['transcription'],
                    })

            data.extend(random.sample(data_, min(len(data_), 30)))
        
    return data

In [11]:
data = loop((files, 0))

  0%|                                                                                                | 0/10 [00:00<?, ?it/s]/home/mesolitica/.local/lib/python3.10/site-packages/malaya_speech/utils/nemo_featurization.py:464: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):
/home/mesolitica/.local/lib/python3.10/site-packages/malaya_speech/nemo/jasper.py:437: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):
/home/mesolitica/.local/lib/python3.10/site-packages/malaya_speech/utils/nemo_featurization.py:464: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):
/home/mesolitica/.local/lib/python3.10/site-packages/malaya_speech/nemo/jasper.

In [12]:
len(data)

496